In [36]:
from mxnet import nd, gpu, cpu
from SkeletonARModel import SkeletonARModel

ctx = gpu(1)

### Binary분류기 
- 5중 분류기에서 out 블럭만 추가해서 답니다

In [37]:
import mxnet.gluon.nn as nn
import mxnet.gluon.rnn as rnn

class BinarySkeletonARModel(SkeletonARModel) :
    
    def __init__(self, **kwargs) :
        super(BinarySkeletonARModel, self).__init__(**kwargs)
        self.out = nn.Sequential()
        self.out.add(
            nn.Dense(units=5, activation='relu'),
            nn.Dense(units=2),
            )
    
    def forward(self, input) :
        '''
        input.shape = (N, frame 수, 34)
        '''
        X = input
        for blk in self.net :
            X = blk(X)
        for blk in self.out :
            X = blk(X)
        return X

### 이미지 자료 불러오기 & 전처리
- fall/no_fall 자료를 불러옵니다

In [38]:
import pickle
with open('./data_pkl/img_data_binary.pkl', 'rb') as f :
    x_data = pickle.load(f)
    y_data = pickle.load(f)
    
max_len = 94
padded = [nd.concat(x, nd.zeros((max_len-x.shape[0], 17, 2)).as_in_context(ctx), dim=0) for x in x_data]
flat = [p.reshape(-1, 34) for p in padded]

x_data = nd.stack(*flat)
y_data = nd.stack(*y_data)
x_data.shape, y_data.shape

((2124, 94, 34), (2124, 1))

### 동영상 자료 불러오기 & 전처리
- 우리가 찍은 영상으로 부터 제작된 포즈입니다

In [43]:
import pickle
with open('./data_pkl/our_data_five.pkl', 'rb') as f :
    x_test = pickle.load(f)
    y_test = pickle.load(f)

max_len = 94
padded = [nd.concat(x, nd.zeros((max_len-x.shape[0], 17, 2)).as_in_context(ctx), dim=0) for x in x_test]
flat = [p.reshape(-1, 34) for p in padded]
x_test = nd.stack(*flat)
y_test = nd.stack(*y_test)

no_fall=(y_test != 0)
y_test = no_fall
x_data = nd.concat(x_data, x_test, dim=0)
y_data = nd.concat(y_data, y_test, dim=0)
x_data.shape, y_data.shape

((2952, 94, 34), (2952, 1))

### 학습과 검증 데이터로 나눕니다

In [44]:
import numpy as np
num_data = len(x_data)
choice_t = np.random.choice(num_data, (int)(num_data*0.80), replace=False)
choice_v = [n for n in range(num_data) if not n in choice_t]

x_train = x_data[choice_t]
y_train = y_data[choice_t]
x_valid = x_data[choice_v]
y_valid = y_data[choice_v]
num_train = len(x_train)
num_valid = len(x_valid)

#x_train.transpose(axes=(1, 0, 2))
#x_valid.transpose(axes=(1, 0, 2))
print(x_train.shape, x_valid.shape, num_data)

(2361, 94, 34) (591, 94, 34) 2952


## 학습

In [50]:
from mxnet import autograd
from mxnet import gluon, init
import mxnet as mx

### 배치로 넣지않고 하나씩 넣는 경우의 accuracy를 정의
- mxnet.metric.Accuracy 모듈이 내놓는 값이 부정확한 것 같아서 시험삼하 만듦

In [53]:
def get_exp():
    count = 0
    for i, x in enumerate(x_test) :
        y_hat = model(x.expand_dims(0))
        if y_hat.argmax(axis=1) == y_test[i] :
            count += 1
    return count / len(x_test) * 100

### 모델 생성
- 모델을 생성합니다
- 가중치를 초기화합니다.
- `BinarySkeletonAR.params` 에는 이중 분류기의 학습된 가중치가 저장되어 있습니다.
- 맨 처음 학습할 때는 5중 분류기의 가중치값을 가져와서 학습했습니다

In [71]:
model = BinarySkeletonARModel()
model.initialize(init.Xavier(), force_reinit=True, ctx=ctx)
#model.load_parameters("./model_params/SkeletonAR.params", allow_missing=True, ignore_extra=True, ctx=ctx)
model.load_parameters("./model_params/BinarySkeletonAR.params", allow_missing=True, ignore_extra=True, ctx=ctx)

Loss = gluon.loss.SoftmaxCrossEntropyLoss(sparse_label=True)
trainer1 = gluon.Trainer(model.net.collect_params(), 'adam', {'learning_rate': 0.003})
trainer2 = gluon.Trainer(model.out.collect_params(), 'adam', {'learning_rate': 0.01})
acc = mx.metric.Accuracy()

> 프리트레인된 부분과 
> 새로 추가된 부분의 학습률을 다르게 하였음

In [72]:
for epoch in range(10000):
    
    for i in range(4) :
        choice = np.random.choice(num_train, int(num_train * 0.25), replace=False)
        
        with autograd.record() :
            y_hat = model(x_train[choice])
            loss = Loss(y_hat, y_train[choice])
        loss.backward()
        trainer1.step(batch_size=len(choice))
        trainer2.step(batch_size=len(choice))
        
    print("\n******Epoch {%d} ******" % epoch)
    print("**loss > {}".format(nd.sum(loss / len(choice)).asscalar()))
        
    if epoch % 5 == 0 :
        try :
            acc.update(preds=y_hat, labels=y_train[choice])
            print("**\t train_acc >> {}".format(acc.get()))

            valid_y_hat = model(x_test)
            valid_loss = Loss(valid_y_hat, y_test)
            print("**\t valid loss >> {}".format(nd.sum(valid_loss / num_valid).asscalar()))

            acc.update(preds=valid_y_hat, labels=y_test)
            print("**\t valid_acc >> {}, ".format(acc.get()))
            
            print("**\t valid_experience >> {}".format(get_exp()))
            print("**\t predicts >> ", nd.argmax(valid_y_hat, axis=1))
            

        except KeyboardInterrupt :
            break
        except :
            print("errors")    


******Epoch {0} ******
**loss > 0.0725853443145752
**	 train_acc >> ('accuracy', 0.9694915254237289)
**	 valid loss >> 0.0204037856310606
**	 valid_acc >> ('accuracy', 0.9800796812749004), 
**	 valid_experience >> 96.1352657004831
**	 predicts >>  
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1

**loss > 0.06366566568613052

******Epoch {22} ******
**loss > 0.0529676117002964

******Epoch {23} ******
**loss > 0.045542456209659576

******Epoch {24} ******
**loss > 0.056563232094049454

******Epoch {25} ******
**loss > 0.09034799039363861
**	 train_acc >> ('accuracy', 0.9823529411764705)
**	 valid loss >> 0.01607893966138363
**	 valid_acc >> ('accuracy', 0.9835657370517928), 
**	 valid_experience >> 99.7584541062802
**	 predicts >>  
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

### 모델 저장

In [69]:
model.save_parameters("./model_params/BinarySkeletonAR.params")

In [70]:
model.load_parameters("./model_params/BinarySkeletonAR.params", ctx=ctx)

### 사진자료 + 영상자료 전체에 대한 정확도

In [63]:
compare = model(x_data).argmax(axis=1) == y_data.squeeze()
compare.sum() / len(compare)


[0.98848236]
<NDArray 1 @gpu(1)>

### 영상자료에 대한 정확도

In [64]:
compare = model(x_test).argmax(axis=1) == y_test.squeeze()
compare.sum() / len(compare)


[0.9975845]
<NDArray 1 @gpu(1)>

### 모델이 내놓은 예측은?

In [65]:
model(x_test).argmax(axis=1)


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

### 실제 레이블은?

In [68]:
y_test.squeeze()


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1